In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import datetime as dt

#import tick customization tools
import matplotlib.ticker as mticks
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

# list of named time zones
import pytz
#all_tzs = pytz.all_timezones
#len(all_tzs)

# import the statsmodels time series analysis api module
# seasonal decompose
import statsmodels.tsa.api as tsa

# get peaks (max point) of each season
from scipy.signal import find_peaks

import holidays
import datetime as dt
from holidays import country_holidays

import glob


import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'holidays'

In [ ]:
q = 'Data/Chicago/Chicago-Crime_*.csv'
chunked_files = glob.glob(q)
# Showing the first 5
chunked_files

In [ ]:
## Loading and Concatenating the list of dfs with 1 line
df_combined = pd.concat([pd.read_csv(file, index_col=0) for file in chunked_files])
df_combined

In [ ]:
df = df_combined.copy()
df.info()

In [ ]:
# Set options
pd.set_option('display.max_columns',100)
# Customize figure style for stakeholder-facing visualizations
plt.style.use(('ggplot','fivethirtyeight'))
sns.set_context('notebook', font_scale=1.2)
plt.rcParams['figure.figsize'] = (12,4)
plt.rcParams['savefig.transparent'] = False
plt.rcParams['savefig.bbox'] = 'tight'

In [ ]:
df.head()

In [ ]:
fmt = '%m/%d/%Y %I:%M:%S %p'

In [ ]:
df['Datetime'] = pd.to_datetime(df['Date'], format=fmt)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.set_index('Datetime')
df.head(2)

In [ ]:
df.drop(columns=['Latitude', 'Longitude', 'Date'], inplace=True)
df.head(2)

In [ ]:
df['Primary Type'].value_counts().index.sort_values()

In [ ]:
df['Primary Type'] = df['Primary Type'].replace({'CRIM SEXUAL ASSAULT': 'CRIMINAL SEXUAL ASSAULT'})

In [ ]:
df['Primary Type'].value_counts().index.sort_values()

In [ ]:
print(df['Primary Type'].unique())
len(df['Primary Type'].unique())

In [ ]:
non_crime = df['Primary Type'].str.startswith('NON')

In [ ]:
df1 = df[~non_crime]
df1

In [ ]:
len(df1['Primary Type'].unique())

In [ ]:
plot_1 = df1['District'].value_counts().sort_values()
plot_1

In [ ]:
plot_1.values.sum()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
plot_1.plot(kind='barh')

plt.xlabel('Count of Crimes')
plt.ylabel('District Number')
plt.title('Crimes by District');

In [ ]:
df2 = df[~non_crime]

In [ ]:
df2['Year'] = df2.index.year

In [ ]:
df2['Year'].value_counts().sort_index(ascending = False)

In [ ]:
## save year counts
year_counts = df2['Year'].value_counts().sort_index()
year_counts.plot(style='o-',grid=True,ylabel='# of Crimes', xlabel='Year')

plt.xticks(np.arange(2001, 2023, step=2));

In [ ]:
df2

In [ ]:
plot_type = df2.groupby('Primary Type').resample('M').size()

In [ ]:
plot_type = plot_type.unstack().T
plot_type

In [ ]:
plot_type = plot_type.fillna(0.0)
plot_type.isna().sum().sum()

In [ ]:
fig, axes = plt.subplots(11, 3, figsize=(15,30))

for indx, crime in enumerate(plot_type.columns):
    row = indx // 3
    col = indx % 3
    ts = plot_type[crime]
    decomp = tsa.seasonal_decompose(ts)
    
    decomp.trend.plot(title=f'{crime} Trend', ax = axes[row, col])
    axes[row, col].set_title(f'Trend {crime}')
plt.tight_layout()

In [ ]:
df4 = df[~non_crime]

In [ ]:
df4['Month'] = df4.index.month
df4['MonthName'] = df4.index.month_name()
df4.head(3)

In [ ]:
# saving lookup for number vs name
unique_months = df4.drop_duplicates(subset=['Month']).sort_values('Month')
month_lookup =dict(zip(unique_months['Month'],unique_months['MonthName']))
month_lookup

In [ ]:
ax = sns.countplot(data=df4, x='MonthName',palette='dark',order=month_lookup.values());
# Rotate xtick labels and align the text to the bar
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right');

In [ ]:
## save month counts
# month_lookup.values() will reorder df['MonthName'].value_counts() by month
month_counts = df4['MonthName'].value_counts()[month_lookup.values()]
# year_counts = df.groupby("Year").size()
ax = month_counts.plot(style='o-',grid=True,
                       ylabel='# of Crimes',
                       xlabel='Month');

In [ ]:
df4

In [ ]:
plot_type4.index

In [ ]:
plot_type4 = df4.groupby('Primary Type')['Month', 'MonthName'].value_counts().sort_index()
plot_type4

In [ ]:
plot_type4_df = plot_type4.to_frame('# of Crimes').reset_index()
plot_type4_df

In [ ]:
g = sns.catplot(data=plot_type4_df, x='MonthName', y='# of Crimes',
                kind='point',aspect=3, height=4,
                hue='Primary Type', col='Primary Type',col_wrap=2,
               sharey=False, sharex=False)
[ax.grid(axis='x') for ax in g.axes];